In [98]:
from pathlib import Path
import pandas as pd
import plotly.express as px


# Load the data into pandas DataFrame
dir_path = Path("CPA_Data_IntroductionToMarketing_2024")
clv_data = pd.read_csv(dir_path / "clv.csv")
filter_user_data = pd.read_csv(dir_path / "filter_user.csv")

print(clv_data.head())
filter_user_data.head()

   cohort  user  time_year  time_month  subscription  content  genres  \
0       0     0       2017           6             1      4.0     2.0   
1       0     0       2017           7             1      7.0     2.0   
2       0     1       2017           6             1      5.0     4.0   
3       0     1       2017           7             1      9.0     4.0   
4       0     2       2017           6             1      7.0     3.0   

   recency_new  bounce  
0       0.1429  0.2857  
1       0.1667  0.2500  
2       0.6000  1.0000  
3       0.8000  0.5000  
4       0.6250  0.6250  


,title,recency_new,duration_long,genre_family_comedy,genre_drama,genre_action_adventure,genre_thriller_crime,genre_documentary
0,6 Days,1,1,0,0,0,0,1
1,8 Mile,0,1,0,1,0,0,0
2,RuPaul's Drag Race Holi-Slay Spectacular,1,0,1,0,0,0,0
3,Russia's Toughest Prisons,0,0,0,0,0,0,1
4,SPF-18,1,0,1,0,0,0,0


In [99]:
# 2a
start = clv_data[clv_data.time_year == 2017].time_month.min()
end = clv_data[clv_data.time_year == 2018].time_month.max()
nunique = clv_data["user"].nunique()
print((start, 2017), (end, 2018), nunique)

(np.int64(6), 2017) (np.int64(5), 2018) 20850


In [100]:
# 3a
print("CLV DATA: " + str(clv_data))
def calculate_cohort_retention_rate(clv_data, cohort):
    # Filter data for the specified cohort
    clv_data_cohort = clv_data[clv_data.cohort == cohort]
    initial_cohort_size = clv_data_cohort.user.nunique()

    # Calculate active users per month for this cohort
    active_users_per_month = clv_data_cohort.groupby(
        ["time_year", "time_month"], as_index=False
    )["user"].nunique()

    # Calculate retention rate over time
    retention_rate_time = pd.DataFrame()
    retention_rate_time["time"] = (
        active_users_per_month[["time_month", "time_year"]]
        .astype(str)
        .agg("/".join, axis=1)
    )
    retention_rate_time["retention_rate"] = (
        active_users_per_month.user / initial_cohort_size
    ) * 100

    return retention_rate_time

retention_rate_time_0 = calculate_cohort_retention_rate(clv_data, 0)
print("Retention rate time:\n" + str(retention_rate_time_0) + "\n\n")


# 3b
def draw_retention_rate_time(retention_rate_time):
    fig = px.line(
        x=retention_rate_time["time"],
        y=retention_rate_time["retention_rate"],
        labels={"x": "Time (Month/Year)", "y": "Retention Rate (%)"},
        title="Retention Rate over Time",
    )
    fig.show()
    retention_rate_time

draw_retention_rate_time(retention_rate_time_0)

# 3c
avg_retention_rate = retention_rate_time_0["retention_rate"].mean()
print("Average retention rate of cohort 0: " + str(avg_retention_rate))


CLV DATA:        cohort   user  time_year  time_month  subscription  content  genres  \
0           0      0       2017           6             1      4.0     2.0   
1           0      0       2017           7             1      7.0     2.0   
2           0      1       2017           6             1      5.0     4.0   
3           0      1       2017           7             1      9.0     4.0   
4           0      2       2017           6             1      7.0     3.0   
...       ...    ...        ...         ...           ...      ...     ...   
34009      11  20845       2018           5             1      3.0     2.0   
34010      11  20846       2018           5             1      6.0     2.0   
34011      11  20847       2018           5             1      4.0     2.0   
34012      11  20848       2018           5             1      2.0     1.0   
34013      11  20849       2018           5             1      4.0     2.0   

       recency_new  bounce  
0           0.1429  0.28

Average retention rate of cohort 0: 15.370633791686418


3d. 

Industry Benchmarks
Netflix and other major Subscription Video On Demand (SVOD) platforms typically report much higher retention rates. Industry data suggests:
1.  Monthly Retention Rates: SVOD platforms like Netflix often have monthly retention rates exceeding 90%.
2. Monthly Churn Rates: Reported churn rates are generally low, often around 2% to 5% per month, implying retention rates of 95% to 98%.


Comparing our calculated retention rates with industry benchmarks:

Our Retention Rates:

- Sharp decline from 100% to 71.65% in the first month.
- Further drop to 9.44% in the second month.
- Retention rates fall below 1% after a few months.


Industry Benchmarks:

- Retention rates remain above 90% month over month.
- Churn rates are low, indicating that most users continue their subscriptions.

Several reasons may cause this discrepancy between our results and industry benchmarks:

1. Data Limitations

    i. Inability to Track All User Activity:

    The data only captures user activity on desktop and laptops.
    Users who switch to other devices (e.g., mobile phones, tablets, smart TVs) are not tracked and are considered as churned in our dataset.

    ii. Subscription Variable:

    The subscription variable is always 1 in the dataset, as we cannot observe if a user cancels their subscription.
    We define churn as users who stop watching Netflix on desktop or laptop, which may not reflect actual cancellations.

2. Overestimation of Churn

    i. Device Switching:

    Users may simply change devices rather than cancel their subscriptions.
    Given the popularity of mobile and smart TV usage for streaming, many users may migrate away from desktops.

    ii. Misclassification of Active Users:

    Users who are inactive on desktop/laptop but active on other devices are misclassified as churned.

3. Data Collection Period and Sample

    i. Sample Representation:

    The dataset may not be representative of the entire Netflix user base.
    It may focus on a specific segment with different usage patterns.

    ii. Time Frame:

    The data covers a specific 12-month period, which may not capture long-term retention behaviors.

4. User Behavior Patterns

    i. Desktop Usage Decline:

    There has been a general trend towards increased mobile and connected TV usage for streaming. Desktop streaming may show higher churn due to this shift.

    ii. Content Consumption Habits:

    Users might use desktops for initial sign-ups or trials and then switch to other devices.



In [101]:
# 3e
for i in range(0, 12):
    retention_rate_time_i = calculate_cohort_retention_rate(clv_data, i)
    print(f"Monthly retention rate for cohort {i}:\n{retention_rate_time_i['retention_rate']}")
    print(f"Average retention rate for cohort {i} is {retention_rate_time_i['retention_rate'].mean()}\n")


# 3f
# Ensure 'time_year' and 'time_month' are integers
clv_data['time_year'] = clv_data['time_year'].astype(int)
clv_data['time_month'] = clv_data['time_month'].astype(int)

# Create 'date' column & convert the string into Pandas datetime object for later calculations
clv_data['date'] = pd.to_datetime(clv_data['time_year'].astype(str) + '-' + clv_data['time_month'].astype(str) + '-01')

# Find the earliest date, to use as reference point
min_date = clv_data['date'].min()

# Calculate 'month_index' as the number of months since 'min_date', * 12 for converting year difference into months, and + month difference
clv_data['month_index'] = (clv_data['date'].dt.year - min_date.year) * 12 + (clv_data['date'].dt.month - min_date.month)

# Calculate 'cohort_month_index' for each cohort , month_index.min is essentially acquisition month , reset index converts back into dataframe , rename to cohort month index for clarity in the new dataframe
cohort_month_index = clv_data.groupby('cohort')['month_index'].min().reset_index().rename(columns={'month_index': 'cohort_month_index'})

# Merge cohort month index back into clv_data , using cohort as the key , left join to keep all rows in clv_data 
clv_data = clv_data.merge(cohort_month_index, on='cohort', how='left')

# Calculate 'month_since_acquisition' using new cohort_month_index column
clv_data['month_since_acquisition'] = clv_data['month_index'] - clv_data['cohort_month_index']

# Define the function to calculate retention rates
def calculate_monthly_cohort_retention_rate(clv_data, cohort):
    # Filter data for the specified cohort
    cohort_data = clv_data[clv_data['cohort'] == cohort]

    # Initial number of users in cohort
    initial_users = cohort_data['user'].nunique()

    # Calculate active users per 'month_since_acquisition'
    users_per_month = cohort_data.groupby('month_since_acquisition')['user'].nunique().reset_index()

    # Calculate retention rate
    users_per_month['retention_rate'] = (users_per_month['user'] / initial_users) * 100

    # Add 'cohort' information
    users_per_month['cohort'] = cohort

    return users_per_month[['cohort', 'month_since_acquisition', 'retention_rate']]

# Initialise empty list to store DataFrames
retention_list = []

# Loop through each cohort and calculate retention rates
for cohort in clv_data['cohort'].unique():
    cohort_retention = calculate_monthly_cohort_retention_rate(clv_data, cohort)
    retention_list.append(cohort_retention)

# Combine all cohorts into a single DataFrame
retention_data = pd.concat(retention_list, ignore_index=True)

# Calculate average retention rate across cohorts
average_retention = retention_data.groupby('month_since_acquisition')['retention_rate'].mean().reset_index()

# Sort by 'month_since_acquisition'
average_retention = average_retention.sort_values('month_since_acquisition')

# Display the average retention rates
print("Average Retention Rate Across Cohorts:")
display(average_retention)


#3g
# Calculate the average retention rate per cohort
average_retention_per_cohort = retention_data.groupby('cohort')['retention_rate'].mean().reset_index()

# Display the average retention rates per cohort
print("\nAverage Retention Rate per Cohort:")
display(average_retention_per_cohort)

# Calculate the overall average retention rate across months
overall_average_retention_rate = average_retention['retention_rate'].mean()
print(f"Overall Average Retention Rate Across Months: {overall_average_retention_rate:.2f}%")
# Calculate the average retention rate per cohort
average_retention_per_cohort = retention_data.groupby('cohort')['retention_rate'].mean().reset_index()
# Sort by retention_rate in descending order
average_retention_per_cohort = average_retention_per_cohort.sort_values(by='retention_rate', ascending=False)

# Check if the third cohort has the highest retention rate
highest_retention_cohort = average_retention_per_cohort.iloc[0]['cohort']

if highest_retention_cohort == 2:
    print("\nThe third cohort (Cohort 2) has the highest average monthly retention rate: TRUE")
else:
    print("\nThe third cohort (Cohort 2) has the highest average monthly retention rate: FALSE")

Monthly retention rate for cohort 0:
0     100.000000
1      71.645330
2       9.435752
3       1.422475
4       0.711238
5       0.331911
6       0.331911
7       0.237079
8       0.189663
9       0.047416
10      0.047416
11      0.047416
Name: retention_rate, dtype: float64
Average retention rate for cohort 0 is 15.370633791686418

Monthly retention rate for cohort 1:
0    100.00
1     54.56
2     17.28
3      2.32
4      0.88
5      0.56
6      0.24
7      0.16
Name: retention_rate, dtype: float64
Average retention rate for cohort 1 is 22.0

Monthly retention rate for cohort 2:
0    100.000000
1     52.515546
2      4.126625
3      1.130582
4      0.395704
5      0.226116
6      0.169587
7      0.113058
8      0.056529
Name: retention_rate, dtype: float64
Average retention rate for cohort 2 is 17.637083097795365

Monthly retention rate for cohort 3:
0    100.000000
1     44.559585
2      5.551443
3      1.998520
4      1.332346
5      0.592154
6      0.444115
7      0.296077
8     

,month_since_acquisition,retention_rate
0,0,100.000000
1,1,57.722305
2,2,8.129728
3,3,1.526043
4,4,0.742584
5,5,0.394163
6,6,0.301333
7,7,0.184218
8,8,0.156083
9,9,0.047416



Average Retention Rate per Cohort:


,cohort,retention_rate
0,0,15.370634
1,1,22.000000
2,2,17.637083
3,3,17.221811
4,4,20.426479
5,5,24.477255
6,6,30.641356
7,7,33.210634
8,8,44.754735
9,9,49.894381


Overall Average Retention Rate Across Months: 14.11%

The third cohort (Cohort 2) has the highest average monthly retention rate: FALSE


3h. 

Cohort-Specific Analysis (3a, 3b and 3c) is best when:
1. You want to understand how retention varies by the month of acquisition
2. Your goal is to identify factors that affect specific cohorts
3. You are aiming to tailor retention strategies to different user groups

Aggregated Analysis (3f and 3g) is best when:
1. You need a general overview of retention trends
2. You are focusing on overall business performance, rather than cohort differences
3. You need a benchmark to assess how effective different retention initiatives are

You can leverage both approaches, by first using an aggregated analysis to establish a baseline and identify general patterns. After that, you can use a cohort-specific analysis to go more granular in discovering insights and tailor your strategies accordingly. An approach like this will allow you to maximise the strengths of both approaches, and also allow for a much better understanding of customer retention.


In [102]:
# 8
filter_user_data.head()

,title,recency_new,duration_long,genre_family_comedy,genre_drama,genre_action_adventure,genre_thriller_crime,genre_documentary
0,6 Days,1,1,0,0,0,0,1
1,8 Mile,0,1,0,1,0,0,0
2,RuPaul's Drag Race Holi-Slay Spectacular,1,0,1,0,0,0,0
3,Russia's Toughest Prisons,0,0,0,0,0,0,1
4,SPF-18,1,0,1,0,0,0,0


In [103]:
# 8c
genre_columns = [
    "genre_family_comedy",
    "genre_drama",
    "genre_action_adventure",
    "genre_thriller_crime",
    "genre_documentary",
]

attributes = ["recency_new", "duration_long"] + genre_columns
conjoint_attribute_level_counts = (
    filter_user_data.groupby(attributes, as_index=False)
    .size()
    .sort_values(by=["size"], ascending=False)
    .rename(columns={"size": "counts"})
)
total_count = conjoint_attribute_level_counts.counts.sum()
conjoint_attribute_level_counts["streaming frequency"] = (
    conjoint_attribute_level_counts.counts / total_count
)

conjoint_attribute_level_counts["genre"] = (
    filter_user_data[genre_columns].idxmax(axis=1).str.replace("genre_", "")
)
conjoint_attribute_level_counts.drop(columns=genre_columns)
# Note all other attribute levels have frequency 0

,recency_new,duration_long,counts,streaming frequency,genre
4,0,1,5,0.25,family_comedy
2,0,1,2,0.10,family_comedy
6,1,0,2,0.10,family_comedy
10,1,1,2,0.10,family_comedy
11,1,1,2,0.10,drama
0,0,0,1,0.05,documentary
1,0,0,1,0.05,drama
3,0,1,1,0.05,documentary
5,1,0,1,0.05,family_comedy
7,1,1,1,0.05,family_comedy


In [104]:
import statsmodels.api as sm

# Independent variables
X = conjoint_attribute_level_counts[attributes]
# Dependend variable
y = conjoint_attribute_level_counts["streaming frequency"]

# Add a constant to the model (intercept)
# X = sm.add_constant(X)

# Perform linear regression
model = sm.OLS(y, X).fit()

# Display the regression results
print(model.summary())

# Extract the part-worths (coefficients)
part_worths = model.params
ranges = {
    "recency_new": abs(part_worths["recency_new"]),
    "duration_long": abs(part_worths["duration_long"]),
    "genre": abs(part_worths[genre_columns].max() - part_worths[genre_columns].min()),
}

total_range = sum(ranges.values())

relative_importance = {
    key: (value / total_range) * 100 for key, value in ranges.items()
}

print("Part-Worths:\n", part_worths)
print("\nRelative Importance [%]:\n", relative_importance)

ModuleNotFoundError: No module named 'scipy.sparse._sparsetools'